In [1]:
from pymongo import MongoClient
from pd_doc2vec import doc2vec
from tqdm import tqdm
import pandas as pd


client = MongoClient()
client = MongoClient('localhost', 27017) 
#Connects to the MongoDB, make sure youre SSH'ed into the docker

collection_list = client['twitter'].collection_names() 
#Scrapes all the collection names from the Docker

sentimentdata = []
for posts in tqdm(collection_list):
    for post in client['twitter'][posts].find({'sentiment': {"$exists": True}}):  
        #Extracts all the entries with a sentiment field
        post.update({"Source": str(posts)}) 
        #Updates each data with the name of the collection it comes from
        sentimentdata.append(post) 

df = pd.DataFrame(sentimentdata)
# pulls data into the Pandas DataFrame


print(df.head(10))

/home/manny/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/manny/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/manny/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  # Remove the CWD from sys.path while we load stuff.
100%|██████████| 19/19 [11:11<00:00, 35.36s/it]

  Lat_long                   Source  \
0        |  LabeledStatuses_Sandy_K   
1        |  LabeledStatuses_Sandy_K   
2        |  LabeledStatuses_Sandy_K   
3        |  LabeledStatuses_Sandy_K   
4        |  LabeledStatuses_Sandy_K   
5        |  LabeledStatuses_Sandy_K   
6        |  LabeledStatuses_Sandy_K   
7        |  LabeledStatuses_Sandy_K   
8        |  LabeledStatuses_Sandy_K   
9        |  LabeledStatuses_Sandy_K   

                                           TweetBody                TweetID  \
0  -1 RT @_dreamvillle: On a scale from 1-5 how s...  |"261938696272629761"   
1  @jenfromTHEbloc Hi! I have you and all your NY...  |"261964535831871488"   
2  #Pakistan Salesforce Social Marketing Cloud Bu...  |"261985749170270208"   
3  Frankenstorm heads toward US East Coast: Hurri...  |"262095362175684609"   
4  5 reasons why Sandy is expected to be a supers...  |"262227769931493376"   
5  Hurricane Sandy is hitting Chick Fil A hard - ...  |"262235426356596736"   
6  If this storm 

In [2]:
print(df.columns.values) #How many Columns we have in our dataframe


['Lat_long' 'Source' 'TweetBody' 'TweetID' 'UserID' '_id' 'classification'
 'date_time' 'notes' 'original_file' 'sentiment' 'text']


In [3]:

df['Source'].value_counts() 

#How many sources our data comes from,  different sources might label the 
#same data under different names/labels

Misc_MiscFilmReviews_M     100000
LabeledStatuses_Sandy_K       325
Name: Source, dtype: int64

In [4]:
print(df.loc[df['Source'] == 'Misc_MiscFilmReviews_M'].sample(4))

# This source contains the text under "text" 


      Lat_long                  Source TweetBody TweetID UserID  \
45179      NaN  Misc_MiscFilmReviews_M       NaN     NaN    NaN   
52630      NaN  Misc_MiscFilmReviews_M       NaN     NaN    NaN   
94812      NaN  Misc_MiscFilmReviews_M       NaN     NaN    NaN   
59330      NaN  Misc_MiscFilmReviews_M       NaN     NaN    NaN   

                            _id classification date_time notes original_file  \
45179  5beee74b8d75df321cfe78c7            NaN       NaN   NaN           NaN   
52630  5beee7988d75df32fe7c3aa1            NaN       NaN   NaN           NaN   
94812  5beee7988d75df32fe7cdf67            NaN       NaN   NaN           NaN   
59330  5beee7988d75df32fe7c54cd            NaN       NaN   NaN           NaN   

      sentiment                                               text  
45179  negative  the story of a woman ann on her death bed her ...  
52630  positive  this movie is based on the true story of iowa ...  
94812  negative  rex reed once said of a movie julia and

In [5]:
print(df.loc[df['Source'] == 'LabeledStatuses_Sandy_K'].sample(4))

# This source contains the text under "TweetBody" and has a "classification"
# field

    Lat_long                   Source  \
215        |  LabeledStatuses_Sandy_K   
150        |  LabeledStatuses_Sandy_K   
93         |  LabeledStatuses_Sandy_K   
5          |  LabeledStatuses_Sandy_K   

                                             TweetBody                TweetID  \
215  RT @EarlDibblesJr: Hurricane Sandy aint nothin...  |"263694939572027392"   
150  Before you RT #photos â†’That amazing #Sandy p...  |"263042770384605185"   
93   RT @NotBranden: NYC/east coast: If cell networ...  |"262958230118989824"   
5    Hurricane Sandy is hitting Chick Fil A hard - ...  |"262235426356596736"   

          UserID                       _id  classification         date_time  \
215  GrantDominy  5c0e7b47d2512c61248b159c  conversational  10/31/2012 17:32   
150   Care_Aware  5c0e7b47d2512c61248b155b  conversational  10/29/2012 22:20   
93    WolfManNYI  5c0e7b47d2512c61248b1522   informational  10/29/2012 16:45   
5        ssjarvs  5c0e7b47d2512c61248b14ca  conversational  10/27/20

In [6]:

df["main"] = df["sentiment"]
# we are going to build a main label that is just the sentiment

df["sentiment"] = df["Source"] + "_" + df["sentiment"]
# We add source informationg with the sentiment

df['text'].fillna(df['TweetBody'], inplace=True)
# we move the text from tweetbody into text

df['classification'] = df['classification'].fillna("0")
#we populate empty classification  fields with "0"


print(df.isna().sum())

df = df.applymap(str)
print(df.isna().sum())



Lat_long          100000
Source                 0
TweetBody         100000
TweetID           100000
UserID            100000
_id                    0
classification         0
date_time         100000
notes             100000
original_file     100000
sentiment              0
text                   0
main                   0
dtype: int64
Lat_long          0
Source            0
TweetBody         0
TweetID           0
UserID            0
_id               0
classification    0
date_time         0
notes             0
original_file     0
sentiment         0
text              0
main              0
dtype: int64


In [ ]:
print("Building model")

# We pass the class 3 fields:
# 1. The DataFrame
# 2. The X value, the Text (Pandas Series)
# 3. The Y values, the labels that correspond to the text (Pandas Series)
#                   It can be a list of the names of the columns or one as a string

x = doc2vec(df.sample(100), "text", ["main",  "Source", 'sentiment', 'classification'])


print("Scoring Model")

#returns scores of each labels accuracy from the first column passed into 
# the Y or 3 arguments, in this example it was "main"
# this uses sklearn.metrics.f1_score, you can pass in 
print(x.score())

print("Predicting on text")

# You can predict text as follows
print(x.predict_text("its going to be the worst time ever"))


In [ ]:
# how to save model from https://radimrehurek.com/gensim/models/doc2vec.html

from gensim.test.utils import get_tmpfile

fname = get_tmpfile("my_doc2vec_model")
x.model.save(fname)
x.model = Doc2Vec.load(fname)  # you can continue training with the loaded model!

# If you’re finished training a model (=no more updates, only querying, reduce memory usage), you can do:

x.model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)



